In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [2]:
class GridSearch():

    def __init__(self, estimator, params, cv=None, refit=True, verbose=10, threads = 1):
        self.model = GridSearchCV(
            estimator=estimator, param_grid=params, cv=cv, refit=refit, verbose=verbose, n_jobs=threads)

    def fit(self, X, Y):
        self.model.fit(X, Y)
        print('Grid Search Done....')

    def predict(self, x):
        return self.model.predict(x)

    def predict_proba(self, x):
        return self.model.predict_proba(x)

    def score(self, X, Y):
        return self.model.score(X, Y)

    def set_params(self, **kwargs):
        self.model.set_params(**kwargs)

    def get_params(self, deep=True):
        return self.model.get_params(deep)

    def params(self):
        return self.model.cv_results_, self.model.best_score_, self.model.best_params_, self.model.best_estimator_


In [3]:
def GridSearchModel(X_matrix, Y, x_matrix, y, parameters, model=None , filename=None, cv=None, threads = 1):
    
    if model is not None:
        gd = GridSearch(model, parameters, cv=cv, threads = threads)
    else:
        pipe = Pipeline([('model', DummyEstimator())]) # Placeholder Estimator
        gd = GridSearch(pipe, parameters, cv=cv, threads = threads)
    
    print('\n--------------------------------FIT---------------------------------------------\n')
    gd.fit(X_matrix, Y)

    print('\n----------------------------Parameters---------------------------------------------\n')
#     training_score = gd.score(X_matrix, Y)
#     print('Trainign Accuracy score : ', training_score)
#     print('\n------------------------------------------------------------------------------------\n')

    y_trainig_hat = gd.predict(X_matrix)
    train_metric = metric_score(Y, y_trainig_hat, type='Training')
    
    print('Training Prediction probability : ', gd.predict_proba(X_matrix))
    print('\n------------------------------------------------------------------------------------\n')

#     test_score = gd.score(x_matrix, y)
#     print('Test Accuracy score : ', test_score)
#     print('\n------------------------------------------------------------------------------------\n')

    y_test_hat = gd.predict(x_matrix)
    test_metric = metric_score(y, y_test_hat, type='Test')
    
    print('Test Prediction probability : ', gd.predict_proba(x_matrix))
    print('\n------------------------------------------------------------------------------------\n')
    
    results, best_score, best_params, best_estimator = gd.params()

    pd_results = pd.DataFrame(results)
    
    print('Results : \n', pd_results)
    print('\n------------------------------------------------------------------------------------\n')
    print('Best score : ', best_score)
    print('\n------------------------------------------------------------------------------------\n')
    print('Best parameters ', best_params)
    print('\n------------------------------------------------------------------------------------\n')
    print('Best estimator ', best_estimator)
    print('\n------------------------------------------------------------------------------------\n')
    
    if filename is not None:
        pd_best = pd.DataFrame({"best_params" : best_params})
        
        save_results_html(filename, pd.concat([pd.concat([train_metric, test_metric], axis=1, sort=False), pd_best], keys=['Score', 'Model'], axis=1, sort=False))
        save_results_csv(filename, pd_results)
        save_results_csv(filename, pd_best, mode='a')
    

In [3]:
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass